In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [2]:
class Contextual_bandit():
    def __init__(self):
        self.state = 0
        #List out bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,0,-5], [0.1,-5,1,0.25], [-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
    
    def getBandit(self):
        self.state = np.random.randint(0, self.num_bandits)
        return self.state
    
    def pullArm(self, action):
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)    #正态分布
        if result > bandit:
            return 1
        else:
            return -1  

In [3]:
class Agent():
    def __init__(self, learning_rate=0.001, state_size=3, action_size=4):
        self.state_in = tf.placeholder(dtype=tf.int32)
        self.weights = tf.Variable(tf.ones([state_size, action_size]))
        self.output = tf.sigmoid(self.weights[self.state_in]) 
        self.chosen_action = tf.argmax(self.output, 0)
        
        self.reward_ = tf.placeholder(dtype=tf.float32)
        self.action_ = tf.placeholder(dtype=tf.int32)
        self.responsible_weight = self.output[self.action_]
        self.loss = -(tf.log(self.responsible_weight) * self.reward_)
        self.opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(self.loss)

In [4]:
tf.reset_default_graph()
cb = Contextual_bandit()
agent =Agent()
total_episodes = 10000
total_reward = np.zeros([cb.num_bandits, cb.num_actions])
e = 0.1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(total_episodes):
        state = cb.getBandit()
        if np.random.rand(1) < e:
            action = np.random.randint(cb.num_actions)
        else:
            action = sess.run(agent.chosen_action, feed_dict={agent.state_in:state})
        reward = cb.pullArm(action)
        
        resp, ww, _ = sess.run([agent.responsible_weight, agent.weights, agent.opt], 
                               feed_dict={agent.action_:action, agent.reward_:reward, agent.state_in:state}) 
        
        total_reward[state,action] += reward
        if i% 500==0:
            print("Episode {}, Running reward : \n{}".format(i+1, total_reward))
            
print(ww)
print("The agent thinks bandit {} is the most promising ......".format(np.argmax(ww,axis=1)+1))

Episode 1, Running reward : 
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
Episode 501, Running reward : 
[[  -3.    0.    0.  130.]
 [   1.  141.   -6.   -1.]
 [ 161.   -5.   -5.   -4.]]
Episode 1001, Running reward : 
[[  -5.    2.   -5.  275.]
 [   1.  319.   -9.    0.]
 [ 305.   -7.   -9.   -6.]]
Episode 1501, Running reward : 
[[  -7.    2.   -9.  432.]
 [  -2.  461.  -12.    1.]
 [ 470.   -9.  -12.  -12.]]
Episode 2001, Running reward : 
[[  -9.    4.   -8.  571.]
 [  -3.  626.  -13.    2.]
 [ 622.  -16.  -17.  -18.]]
Episode 2501, Running reward : 
[[  -9.    3.   -8.  704.]
 [  -2.  785.  -16.   -1.]
 [ 794.  -22.  -21.  -24.]]
Episode 3001, Running reward : 
[[ -10.    2.   -9.  859.]
 [  -3.  947.  -15.   -5.]
 [ 951.  -24.  -24.  -28.]]
Episode 3501, Running reward : 
[[  -11.     2.    -9.  1008.]
 [   -4.  1106.   -18.    -7.]
 [ 1095.   -36.   -32.   -33.]]
Episode 4001, Running reward : 
[[  -12.     3.    -3.  1142.]
 [   -5.  1276.   -21.   -12.]
 [ 1242.  